In [8]:
import numpy as np

In [9]:
class Game:
    '''
    Game of Minesweeper
    '''
    def __init__(self, n, m):
        '''
        Initialize the board
        
        n (int) - dimensions of board square
        m (int) - number of mines on the board
        board (nxn array) - the actual board
        stateboard (nxn array) - the board the player sees during play
        
        -1 is a mine
        -9 is an uncovered spot on player board
        '''
        self.n = n
        self.m = m
        self.board = np.zeros((self.n, self.n), dtype = int)
        self.stateboard = np.zeros((self.n, self.n), dtype = int)
        self.stateboard.fill(-9)
        
    def rowcol(self, num):
        '''
        Changes number index to row and column index
        '''
        row = num // self.n
        col = int(np.round(((num/self.n) - row) * self.n))
        return row, col
    
    def index(self, row, col):
        '''
        Changes row and column to number index
        '''
        return row * self.n + col
    
    
    def initMines(self, firstClick = None):
        '''
        Initiates mines either before or after the first click
        '''
        
        def fillnumbers(row, col):
            '''
            Adds 1 to all squares around the mine if they should be added
            '''
            for r in [row - 1, row, row + 1]:
                for c in [col - 1, col, col + 1]:
                    if r >= 0 and r < self.n and c >= 0 and c < self.n and not(r == row and c == col):
                        if(self.board[r][c] != -1):
                            self.board[r][c] += 1

        def createBoard():
            # create board based on mines
            for mine in self.mines:
                row, col = self.rowcol(mine)
                self.board[row, col] = -1
            for row in range(self.n):
                for col in range(self.n):
                    if(self.board[row][col] == -1):
                        fillnumbers(row, col)    
        
        if(firstClick):
            row, col = self.rowcol(firstClick)
            # create list of all spots in a 3x3 grid around the first click - these cannot contain mines
            nomines = []
            for i in [-1, 0, 1]:
                for j in [-1, 0, 1]:
                    if(row + i >= 0 and row + i < self.n and col + j >= 0 and col + j < self.n):
                        nomines.append(self.index(row + i, col + j))
                        
            # create an array of all spots that could contain mines - spots in a 3x3 grid around the first click are removed            
            potentialSpots = np.delete(np.array(range(self.n**2)), nomines)
            
            # randomly choose spots for the mines from the potential spots
            self.mines = np.random.choice(potentialSpots, self.m, replace = False)
            createBoard()
            return firstClick
        else:
            # if you want to initiate mines before the first click
            self.mines = np.random.choice(np.array(range(self.n**2)), self.m, replace = False)
            createBoard()
            return None
    
    def showAdjacentNumbers(self, row, col, visited = set()):
        '''
        shows all adjacent zeros until they hit a number in the stateboard - recursive search
        '''
        # don't revisit the same tile
        if (row, col) in visited:
            return
        visited.add((row, col))
        
        # stop searching if you go out of bounds or hit a mine
        if(row < 0 or row >= self.n or col < 0 or col >= self.n):
            return
        if(self.board[row][col] == -1):
            return
        
        # show the zero in the stateboard and continue searching all around
        elif(self.board[row][col] == 0):
            self.stateboard[row][col] = 0
            for i in [-1, 0, 1]:
                for j in [-1, 0, 1]:
                    if not(j == 0 and i == 0):
                        self.showAdjacentNumbers(row + i, col + j, visited)
        # show the number and discontinue searching if you hit a number                 
        else:
            self.stateboard[row][col] = self.board[row][col]
        
    
    def action(self, click):
        '''
        Execute whats happens after someone clicks - click is given as the index integer
        Return 1 if you hit a mine, 0 otherwise
        '''
        row, col = self.rowcol(click)
        
        # die if u hit the mine --- 0 is inplay, 1 is die, 2 is game win
        if(self.board[row][col] == -1):
            return 1
        
        # just show the one you clicked if there is a mine nearby
        if(self.board[row][col] > 0):
            self.stateboard[row][col] = self.board[row][col]
            return 0
        
        # show all blank spaces adjacent if you clicked a 0
        if(self.board[row][col] == 0):
            self.showAdjacentNumbers(row, col)
            return 0

    def human_play(self, firstClick = None):
        '''
        Executes human gameplay --- can change action selection function to be separate... combine human/robot here
        '''
        def select_action():
            '''
            Prints the board and asks the player to select an action
            '''
            print(self.stateboard)
            print('Please click on a tile')
            action = int(input('index: '))
            # while action in self.stateboard.where(!= -9) --- redo action if the board alrdy seen, or not possible action:
            #    pass
            return action
        start = self.initMines(firstClick)
        if(start):
            result = self.action(start)
        else:
            start = select_action()
            result = self.action(start)
            
        # keep playing while result == 0
        while result == 0:
            click = select_action()
            result = self.action(click)
            # if result != -1 and there are no more -9, result = 2
        if result == 1:
            print('You Lose')
        elif(result == 2):
            print('You Win!!!!')
        else:
            print('error: result should be 1 or 2')


In [10]:
x = Game(10, 10)
x.human_play(16)


[[-9 -9 -9  1  0  0  0  0  0  0]
 [-9 -9 -9  1  0  0  0  0  0  0]
 [-9 -9 -9  1  0  0  0  1  1  1]
 [-9 -9 -9  1  0  1  1  2 -9 -9]
 [-9 -9 -9  2  2  4 -9 -9 -9 -9]
 [-9 -9 -9 -9 -9 -9 -9 -9 -9 -9]
 [-9 -9 -9 -9 -9 -9 -9 -9 -9 -9]
 [-9 -9 -9 -9 -9 -9 -9 -9 -9 -9]
 [-9 -9 -9 -9 -9 -9 -9 -9 -9 -9]
 [-9 -9 -9 -9 -9 -9 -9 -9 -9 -9]]
Please click on a tile
index: 12
[[-9 -9 -9  1  0  0  0  0  0  0]
 [-9 -9  1  1  0  0  0  0  0  0]
 [-9 -9 -9  1  0  0  0  1  1  1]
 [-9 -9 -9  1  0  1  1  2 -9 -9]
 [-9 -9 -9  2  2  4 -9 -9 -9 -9]
 [-9 -9 -9 -9 -9 -9 -9 -9 -9 -9]
 [-9 -9 -9 -9 -9 -9 -9 -9 -9 -9]
 [-9 -9 -9 -9 -9 -9 -9 -9 -9 -9]
 [-9 -9 -9 -9 -9 -9 -9 -9 -9 -9]
 [-9 -9 -9 -9 -9 -9 -9 -9 -9 -9]]
Please click on a tile
index: 2
You Lose
